In [10]:
import pulp

# Create a linear programming problem
prob = pulp.LpProblem("BMC_Transportation", pulp.LpMinimize)

# Define decision variables
x = pulp.LpVariable.dicts("x", ((i, j) for i in ['Newark', 'Jacksonville'] for j in ['Boston', 'Columbus', 'Atlanta', 'Richmond', 'Mobile']), lowBound=0, cat='Integer')

# Updated costs from the given data
costs = {
    ('Newark', 'Boston'): 30,
    ('Newark', 'Columbus'): 80,
    ('Newark', 'Atlanta'): 115,
    ('Newark', 'Richmond'): 40,
    ('Newark', 'Mobile'): 150,
    ('Jacksonville', 'Columbus'): 85,
    ('Jacksonville', 'Atlanta'): 45,
    ('Jacksonville', 'Richmond'): 50,
    ('Jacksonville', 'Mobile'): 50
}

# Define the objective function
prob += pulp.lpSum([x[i, j] * costs[i, j] for i, j in x if (i, j) in costs])

# Define the constraints

# Supply constraints
prob += pulp.lpSum([x['Newark', j] for j in ['Boston', 'Columbus', 'Atlanta', 'Richmond', 'Mobile'] if ('Newark', j) in costs]) == 200, "Supply_Newark"
prob += pulp.lpSum([x['Jacksonville', j] for j in ['Columbus', 'Atlanta', 'Richmond', 'Mobile'] if ('Jacksonville', j) in costs]) == 300, "Supply_Jacksonville"

# Demand constraints
prob += pulp.lpSum([x[i, 'Boston'] for i in ['Newark', 'Jacksonville'] if (i, 'Boston') in costs]) == 100, "Demand_Boston"
prob += x['Newark', 'Columbus'] == 20, "Enforced_Newark_to_Columbus"
prob += x['Jacksonville', 'Columbus'] == 40, "Enforced_Jacksonville_to_Columbus"
prob += pulp.lpSum([x[i, 'Atlanta'] for i in ['Newark', 'Jacksonville'] if (i, 'Atlanta') in costs]) == 170, "Demand_Atlanta"
prob += pulp.lpSum([x[i, 'Richmond'] for i in ['Newark', 'Jacksonville'] if (i, 'Richmond') in costs]) == 80, "Demand_Richmond"
prob += pulp.lpSum([x[i, 'Mobile'] for i in ['Newark', 'Jacksonville'] if (i, 'Mobile') in costs]) == 70, "Demand_Mobile"

# Solve the problem
prob.solve()

# Display the solution
for v in prob.variables():
    if v.varValue > 0:  # Only print non-zero decisions for clarity
        print(v.name, "=", v.varValue)

# Display the minimized cost
print("Total Cost = ", pulp.value(prob.objective))


x_('Jacksonville',_'Atlanta') = 170.0
x_('Jacksonville',_'Columbus') = 40.0
x_('Jacksonville',_'Mobile') = 70.0
x_('Newark',_'Boston') = 100.0
x_('Newark',_'Columbus') = 20.0
x_('Newark',_'Richmond') = 80.0
Total Cost =  22350.0


In [12]:
print("Shadow Prices/Dual Values:")
for name, constraint in prob.constraints.items():
    print(f"{name}: {constraint.pi}")


Shadow Prices/Dual Values:
Supply_Newark: -10.0
Supply_Jacksonville: 0.0
Demand_Boston: 40.0
Enforced_Newark_to_Columbus: 90.0
Enforced_Jacksonville_to_Columbus: 85.0
Demand_Atlanta: 45.0
Demand_Richmond: 50.0
Demand_Mobile: 50.0


In [13]:
print("\nSlack Values:")
for name, constraint in prob.constraints.items():
    print(f"{name}: {constraint.slack}")



Slack Values:
Supply_Newark: -0.0
Supply_Jacksonville: 20.0
Demand_Boston: -0.0
Enforced_Newark_to_Columbus: -0.0
Enforced_Jacksonville_to_Columbus: -0.0
Demand_Atlanta: -0.0
Demand_Richmond: -0.0
Demand_Mobile: -0.0
